# Imports

In [105]:
import pandas as pd

## Globals

In [106]:
year = 2024

## Code

In [107]:
# Import song list from csv
raw_list = pd.read_csv("./Data/%s/%s-first-do-not-play.csv" % (year, year))

In [108]:
# Remove extra header lines
rows_to_drop = raw_list[(raw_list['Year'] == 'Year') &
                        (raw_list['Artist'] == 'Artist') &
                        (raw_list['Song Title'] == 'Song Title') &
                        (raw_list['Reason'] == 'Reason')].index
df = raw_list.drop(rows_to_drop)

In [109]:
# Fill all empty reasons with n/a
df['Reason'] = df['Reason'].fillna('n/a')
# Get list of unique reasons
unique_reasons = df['Reason'].unique().tolist()
print(unique_reasons)

['Language', 'Language / Content', 'Sexual Content', 'Drinking / Drugs / Partying', 'Violence', 'Content - Sad', 'Content - Sensitive Topic', 'content - Sad', 'language', 'Religion', 'Content - sad', 'n/a', 'content - sad', 'Language/content - sad', 'language / Content', 'Language / content', 'Reason', 'Drinking / Drugs / P', 'language / content']


In [110]:
# Fix the songs that have their reason in the song title
# List of phrases to search for at the end of Song Title
reasons = ['Language',
            'Language / Content',
            'Sexual Content',
            'Drinking / Drugs / Partying',
            'Violence',
            'Content - Sad',
            'Content - Sensitive Topic',
            'content - Sad',
            'language',
            'Religion',
            'Content - sad',
            'content - sad',
            'Language/content - sad',
            'language / Content',
            'Language / content',
            'Reason',
            'Drinking / Drugs / P',
            'language / content',
            'n/a']

# Function to modify 'Reason' and 'Song Title' based on conditions
def fix_null_reasons(row):
    if row['Reason'] == 'n/a':  # Check if Reason is n/a
        for phrase in reasons:  # Check if any phrase is at the end of Song Title
            if row['Song Title'].endswith(phrase):
                # Remove the phrase from Song Title
                new_song_title = row['Song Title'].replace(phrase, '').strip()
                # Replace 'n/a' with the found phrase in 'Reason'
                return pd.Series([phrase, new_song_title], index=['Reason', 'Song Title'])
    return row  # Return the row unchanged if no match

# Apply the function to each row
df[['Reason', 'Song Title']] = df.apply(fix_null_reasons, axis=1)

# Display the updated DataFrame
print(df)

AttributeError: 'float' object has no attribute 'endswith'

In [ ]:
# Combine redundant reasons and standardize naming scheme
df.rename(columns={'Song Title': 'Title'}, inplace=True)
df['Reason'] = df['Reason'].str.lower()
df['Reason'] = df['Reason'].replace('reason', 'n/a')
df['Reason'] = df['Reason'].replace('drinking / drugs / p', 'drinking / drugs / partying')
df['Reason'] = df['Reason'].replace('sexual content', 'content - sexual')
df['Reason'] = df['Reason'].replace('language / content', 'language')
df['Reason'] = df['Reason'].replace('language/content - sad', 'content - sad')

In [ ]:
# Save cleaned data. Can be used with the CSV to Spotify app
df.to_csv('./Data/%s/%s_clean.csv' % (year, year), index=False)

In [ ]:
# NEED TO DROP REASON AND YEAR or spotify playlist converter might get confused
# also get rid of anything related to feat. or ft./ or whatever from all columns and maybe if it has a space after?
# use the cleaned data that i manually fixed a few things
# look at all the nas and stuff
# if it's n/a, that's cuz it got the tag inside the name of the song, so try to find the keys and switch them?